In [1]:
?print

In [40]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"
# %whos

# 句子长度统计

In [8]:
# coding=utf-8
import json
import os
import re
import sys

import jieba
from tqdm import tqdm
import pandas as pd

sys.path.append("..")

from utils import dir_op


def clean_text(text:str):
    """Clean text by removing unnecessary characters and altering the format of words.
    nlg 任务的时候尽量保持原有的句子结构，不要去除停用词和标点，否则直接影响流畅程度
    """
    text = re.sub(r"\s", "", text)
    text = " ".join(jieba.cut(text))

    return text


def main():
    root_dir = "data/raw_data/baike2018qa/"

    dir_op.create_dirs([save_dir])

    file_list = ["baike_qa_train.json", "baike_qa_valid.json"]
    files = [os.path.join(root_dir, file) for file in file_list]
    
    df = pd.DataFrame(columns=["question", "answering"])

    for file in files:
        with open(file, "r") as fr:
            for line in tqdm(fr.readlines(), desc="QA数据", colour="green"):
                tmp = json.loads(line)
                question = clean_text(tmp["title"])
                answering = clean_text(tmp["answer"])
                if len(question) > 0 and len(answering) > 0:
                    df = df.append([{"question": question, "answering": answering}], ignore_index=True)
    
    df.to_csv()


if __name__ == "__main__":
    main()


QA数据:  12%|█▏        | 173186/1425170 [21:11<2:33:11, 136.21it/s]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df['text_len'] = df['text'].apply(lambda x: len(x.split(" ")))
print(df['text_len'].describe())

In [ ]:
_ = plt.hist(df['text_len'], bins=100) # bins 数据划分区间
plt.xlabel('Text char count')
plt.title("Histogram of char count")

# 词频统计

使用gensim训练好的的模型中含有vocab，是按照词频排序的

# 词云展示

In [5]:
import wordcloud
import jieba
import os

save_dir = "data/imgs"
font_path="/System/Library/fonts/PingFang.ttc"

txt = "一百年前，中国人苦苦寻找适合中国国情的道路。君主立宪制、复辟帝制、议会制、多党制、总统制都想过了、试过了，结果都行不通。最后，中国选择了社会主义道路"
seg_list = " ".join(jieba.cut(txt))
w=wordcloud.WordCloud(font_path=font_path)
w.generate(seg_list)
w.to_file(os.path.join(save_dir, "cloud.png"))

In [8]:
from gensim.models import Word2Vec

In [9]:
model = Word2Vec.load("checkpoints/word2vec/word2vec.model")

In [13]:
# 获取所有词
vocab = model.wv.index2word
# 获取所有向量
word_vectors = model.wv.vectors

In [37]:
id2vec = model.wv.vectors.tolist()
id2word = model.wv.index2word
vocab = ["<PAD>", "<S>", "<E>", "<UNK>"]
vectors = []

for i in range(len(vocab)):
    vectors.append(list(np.random.uniform(low=-0.1, high=0.1, size=(300,))))

vocab = vocab + id2word
vectors = vectors + id2vec

In [44]:
np.savetxt("vectors.txt", vectors)

In [4]:
word2idx = {word: idx for idx, word in enumerate(vocab)}
idx2word = {idx: word for idx, word in enumerate(vocab)}

In [5]:
import tensorflow as tf

In [6]:
word_embedding = tf.get_variable("word_embedding", dtype=tf.float32,
                                 initializer=tf.constant(word_vector, dtype=tf.float32))

In [7]:
x = tf.nn.embedding_lookup(word_embedding, 0)

In [9]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    z = sess.run(x)

In [3]:
import numpy as np

In [7]:
len(list(np.random.uniform(low=-0.1, high=0.1, size=))

SyntaxError: unexpected EOF while parsing (<ipython-input-7-32cff6d290d1>, line 1)